In [ ]:
import psycopg2
from psycopg2 import sql

conn = psycopg2.connect(
    host="localhost", # Bắt buộc kết nối qua mạng TCP/IP
    dbname="postgres", 
    user="postgres", 
    password="postgres_password"
)

cur = conn.cursor()

In [79]:
import torch
from transformers import AutoModel, AutoTokenizer
from torch.nn.functional import cosine_similarity
import pandas as pd
import re
import os
# Thêm import cần thiết cho tính toán Euclid
import torch.nn.functional as F

# Đặt giới hạn an toàn cho PhoBERT-base
SAFE_MAX_LENGTH = 256 # Giới hạn 256 token là an toàn nhất cho PhoBERT-base
# Giữ 514 cho phần kết hợp QA vì mô hình có thể chấp nhận nếu đã cắt ngắn, 
# nhưng 256 là an toàn hơn. Tuy nhiên, tôi sẽ giữ 514 ở đây để giữ tính đồng nhất với logic gốc.
QA_MAX_LENGTH = 256

# Hàm nhúng (đã cập nhật)
def get_embedding(text, model, tokenizer, max_len=512):
    # Đảm bảo tensor luôn ở device của model
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=max_len).to(model.device)
    with torch.no_grad():
        out = model(**inputs)
    return out.last_hidden_state[:, 0, :] # CLS token

# Hàm tính khoảng cách Euclid giữa một vector và nhiều vector
def euclidean_distance(query_emb, context_embeddings):
    # query_emb: [1, hidden_size]
    # context_embeddings: [n_chunks, hidden_size]
    
    # Mở rộng kích thước của query để phù hợp với context_embeddings
    # Thao tác này tương đương với tính (context_embeddings - query_emb).norm(p=2, dim=1)
    
    # torch.cdist là cách hiệu quả và trực tiếp để tính khoảng cách Euclid
    # giữa hai tập hợp vector. Tuy nhiên, đối với một query so với nhiều context,
    # cách tính norm sau khi trừ là đơn giản và rõ ràng hơn.
    
    # Mở rộng query_emb để có kích thước [n_chunks, hidden_size]
    query_expanded = query_emb.expand_as(context_embeddings)
    
    # Tính hiệu số, sau đó tính chuẩn L2 (Euclidean norm) trên dimension vector (dim=1)
    distances = torch.norm(context_embeddings - query_expanded, p=2, dim=1)
    
    return distances.squeeze() # Trả về tensor 1D chứa các khoảng cách

# ===== 1. Load PhoBERT (Đã sửa đổi) =====
model_name = "vinai/phobert-base"

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
phobert = AutoModel.from_pretrained(model_name, trust_remote_code=True)
try: 
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    phobert.to(device)
    print(f"Model đang chạy trên thiết bị: {device}")
except Exception as e:
    print(f"Cảnh báo: Không thể chuyển model sang GPU: {e}")
    phobert.to("cpu")

# ===== 3. Đọc file câu hỏi =====
csv_path = "training_input/question.csv"
df = pd.read_csv(csv_path)

# ===== 4. Dự đoán =====
for idx, row in df.iterrows():
    question = str(row["Question"]).strip()
    options = [str(row["A"]), str(row["B"]), str(row["C"]), str(row["D"])]
    labels = ["A", "B", "C", "D"]
    
    # 💥 FIX: Đặt max_length an toàn cho câu hỏi
    question_embedding = get_embedding(question, phobert, tokenizer, max_len=512)
    
    if question_embedding is None:
        continue # Bỏ qua câu hỏi rỗng
        
    # Truy vấn DB (không thay đổi)
    cur.execute("""SELECT document FROM dbvector1 
                 Order by embedding  <=> %s :: vector asc
                 LIMIT 1;""", (question_embedding.squeeze().detach().cpu().numpy().tolist(),))
    results = cur.fetchall()
    x = results[0]
    document_name = x[0]
    print(f"\n Câu hỏi: {question}")
   
    # Đọc và chia đoạn tài liệu (logic chia theo 256 ký tự được giữ nguyên)
    md_path = os.path.join("training_output", document_name, "main.md")
    if not os.path.exists(md_path): continue
    
    with open(md_path, "r", encoding="utf-8") as f:
        doc_text = f.read()

    # 🚨 CẢNH BÁO: Logic loại bỏ dấu cách (re.sub) vẫn được giữ nguyên theo yêu cầu,
    # nhưng nó làm giảm đáng kể chất lượng kết quả nhúng.
    processed_text = re.sub(r'\s+', '', doc_text.strip()) 
    doc_chunks = []
    MAX_CHARS = 300 # Giữ 500 ký tự
    for i in range(0, len(processed_text), MAX_CHARS):
        chunk = processed_text[i:i + MAX_CHARS]
        doc_chunks.append(chunk)

    # Tạo embedding cho từng đoạn
    chunk_embeddings = []
    for chunk in doc_chunks:
        # 💥 FIX: Đặt max_length an toàn cho chunk
        emb = get_embedding(chunk, phobert, tokenizer, max_len=300)
        if emb is not None:
            chunk_embeddings.append(emb)

    if not chunk_embeddings:
        print(f"Không tìm thấy chunk hợp lệ trong tài liệu {document_name}. Bỏ qua.")
        continue

    chunk_embeddings = torch.cat(chunk_embeddings, dim=0)  # [n_chunks, hidden_size]

    distances = []
    for i, opt in enumerate(options):
        # *Tối ưu hóa: Nhúng chỉ tùy chọn, so sánh với tất cả các đoạn.*
        qa_text = f"{question} {opt}"
        
        # 💥 FIX: Đặt max_length an toàn cho QA text
        qa_emb = get_embedding(qa_text, phobert, tokenizer, max_len=512)

        if qa_emb is None:
            min_dist = float('inf')
        else:
            # 🔄 THAY ĐỔI LỚN: Tính khoảng cách Euclid (L2 Distance)
            dists = euclidean_distance(qa_emb, chunk_embeddings)
            
            # ➡️ Tìm khoảng cách NHỎ NHẤT (vì khoảng cách Euclid: Càng nhỏ càng tốt)
            min_dist = dists.min().item()
            
        distances.append(min_dist)
        # In ra khoảng cách thay vì similarity
        print(f"  {labels[i]}. {opt}  →  distance = {min_dist:.4f}")

    # ➡️ THAY ĐỔI LỚN: Tìm chỉ mục có khoảng cách NHỎ NHẤT (argmin)
    best_idx = torch.argmin(torch.tensor(distances)).item()
    print(f"👉 Đáp án gợi ý: {labels[best_idx]} - {options[best_idx]}")
    

cur.close()
conn.close()

Cảnh báo: Không thể chuyển model sang GPU: CUDA error: device-side assert triggered
Search for `cudaErrorAssert' in https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__TYPES.html for more information.
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


 Câu hỏi: Trong mô hình nhà thông minh, IoT chủ yếu đóng vai trò gì?
  A. Lưu trữ dữ liệu trên máy chủ  →  distance = 9.8515
  B. Kết nối Internet và quản lý thiết bị từ xa  →  distance = 9.5775
  C. Cung cấp dịch vụ phân tích dữ liệu lớn  →  distance = 10.6536
  D. Thay thế hoàn toàn điện toán đám mây  →  distance = 9.8783
👉 Đáp án gợi ý: B - Kết nối Internet và quản lý thiết bị từ xa

 Câu hỏi: Năm 2010, công nghệ nào thường được sử dụng trong kiểm soát ra vào ngôi nhà thông minh?
  A. Bluetooth Low Energy  →  distance = 10.9916
  B. RFID

KeyboardInterrupt: 

In [64]:
conn.rollback()